En esta clase aprenderá cómo crear y optimizar modelos con aumento de gradiente. 

## Introducción

Durante gran parte de este curso, ha realizado predicciones con el método de bosque aleatorio, que logra un mejor rendimiento que un único árbol de decisión simplemente promediando las predicciones de muchos árboles de decisión.

Nos referimos al método del bosque aleatorio como un "método de conjunto". Por definición, los métodos de conjunto combinan las predicciones de varios modelos (por ejemplo, varios árboles, en el caso de bosques aleatorios).

A continuación, aprenderemos sobre otro método de conjunto llamado aumento de gradiente (gradient boosting).

## Aumento de gradiente

El aumento de gradiente es un método que pasa por ciclos para agregar modelos de forma iterativa a un conjunto.

Comienza inicializando el conjunto con un único modelo, cuyas predicciones pueden ser bastante ingenuas. (Incluso si sus predicciones son tremendamente inexactas, las adiciones posteriores al conjunto abordarán esos errores).

Luego, iniciamos el ciclo:

* Primero, utilizamos el conjunto actual para generar predicciones para cada observación en el conjunto de datos. Para hacer una predicción, sumamos las predicciones de todos los modelos del conjunto.

* Estas predicciones se utilizan para calcular una función de pérdida (como el error cuadrático medio, por ejemplo).

* Luego, usamos la función de pérdida para ajustar un nuevo modelo que se agregará al conjunto. Específicamente, determinamos los parámetros del modelo para que agregar este nuevo modelo al conjunto reduzca la pérdida. (Nota al margen: el "gradiente" en "aumento de gradiente" se refiere al hecho de que usaremos el descenso de gradiente en la función de pérdida para determinar los parámetros en este nuevo modelo).

* Finalmente, agregamos el nuevo modelo al conjunto, y...

* ... ¡repetir!

### Ejemplo
Comenzamos cargando los datos de entrenamiento y validación en X_train, X_valid, y_train e y_valid.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('/kaggle/input/melbourne-housing-snapshot/melb_data.csv')

# Select subset of predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Select target
y = data.Price

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

En este ejemplo, trabajará con la biblioteca XGBoost. XGBoost significa aumento de gradiente extremo, que es una implementación de aumento de gradiente con varias características adicionales centradas en el rendimiento y la velocidad. (Scikit-learn tiene otra versión de aumento de gradiente, pero XGBoost tiene algunas ventajas técnicas).

En la siguiente celda de código, importamos la API scikit-learn para XGBoost (xgboost.XGBRegressor). Esto nos permite construir y ajustar un modelo tal como lo haríamos en scikit-learn. Como verá en el resultado, la clase XGBRegressor tiene muchos parámetros ajustables; ¡pronto aprenderá sobre ellos!

In [3]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [11]:
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 232359.80004602356


## Ajuste de parámetros

XGBoost tiene algunos parámetros que pueden afectar drásticamente la precisión y la velocidad del entrenamiento. Los primeros parámetros que debes entender son:

n_estimadores
n_estimators especifica cuántas veces se debe pasar por el ciclo de modelado descrito anteriormente. Es igual al número de modelos que incluimos en el conjunto.

* Un valor demasiado bajo provoca un ajuste insuficiente, lo que conduce a predicciones inexactas tanto en los datos de entrenamiento como en los de prueba.

* Un valor demasiado alto provoca un sobreajuste, lo que provoca predicciones precisas en los datos de entrenamiento, pero predicciones inexactas en los datos de prueba (que es lo que nos importa).

Los valores típicos oscilan entre 100 y 1000, aunque esto depende mucho del parámetro learning_rate que se analiza a continuación.

Aquí está el código para establecer el número de modelos en el conjunto:

In [12]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

### early_stopping_rounds

"early_stopping_rounds" ofrece una forma de encontrar automáticamente el valor ideal para n_estimators. La parada anticipada hace que el modelo deje de iterar cuando la puntuación de validación deja de mejorar, incluso si no estamos en la parada brusca de n_estimators. Es inteligente establecer un valor alto para n_estimators y luego usar early_stopping_rounds para encontrar el momento óptimo para dejar de iterar.

Dado que el azar a veces provoca una sola ronda en la que las puntuaciones de validación no mejoran, es necesario especificar un número de rondas de deterioro directo que se permitirán antes de detenerse. Establecer early_stopping_rounds=5 es una opción razonable. En este caso, nos detenemos después de 5 rondas seguidas de deterioro de las puntuaciones de validación.

Al utilizar early_stopping_rounds, también debe reservar algunos datos para calcular las puntuaciones de validación; esto se hace configurando el parámetro eval_set.

Podemos modificar el ejemplo anterior para incluir la parada anticipada:

In [15]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)],
             verbose=False)

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

Si más adelante desea ajustar un modelo con todos sus datos, establezca n_estimators en cualquier valor que considere óptimo cuando se ejecute con parada anticipada.

### tasa de aprendizaje
En lugar de obtener predicciones simplemente sumando las predicciones de cada modelo componente, podemos multiplicar las predicciones de cada modelo por un número pequeño (conocido como tasa de aprendizaje) antes de agregarlas.

Esto significa que cada árbol que agregamos al conjunto nos ayuda menos. Entonces, podemos establecer un valor más alto para n_estimators sin sobreajustar. Si utilizamos la parada anticipada, el número apropiado de árboles se determinará automáticamente.

En general, una tasa de aprendizaje pequeña y una gran cantidad de estimadores producirán modelos XGBoost más precisos, aunque también llevará más tiempo entrenar el modelo, ya que realiza más iteraciones a lo largo del ciclo. Por defecto, XGBoost establece learning_rate=0.1.

Modificar el ejemplo anterior para cambiar la tasa de aprendizaje produce el siguiente código

In [16]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

n_trabajos

En conjuntos de datos más grandes donde se considera el tiempo de ejecución, puede utilizar el paralelismo para construir sus modelos más rápido. Es común establecer el parámetro n_jobs igual al número de núcleos de su máquina. En conjuntos de datos más pequeños, esto no ayudará.

El modelo resultante no será mejor, por lo que la microoptimización para el tiempo de adaptación no suele ser más que una distracción. Sin embargo, es útil en conjuntos de datos grandes donde, de otro modo, pasaría mucho tiempo esperando durante el comando de ajuste.

Aquí está el ejemplo modificado:

In [17]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=4,
             num_parallel_tree=None, random_state=None, ...)

Conclusión

XGBoost es una biblioteca de software líder para trabajar con datos tabulares estándar (el tipo de datos que almacena en Pandas DataFrames, a diferencia de tipos de datos más exóticos como imágenes y videos). Con un ajuste cuidadoso de los parámetros, puede entrenar modelos de alta precisión.